In [230]:
import os
import pandas as pd
from getpass import getpass
from sklearn import tree
from sklearn.model_selection import train_test_split
import bentoml

In [233]:
# Please set these variables in your local notebook (if you want to run this locally)
# os.environ['YATAI_HOST']="a64acbaff7bb1491d88bc34071c375c1-663950777.eu-west-3.elb.amazonaws.com"

In [231]:
if "YATAI_TOKEN" not in os.environ.keys() or os.environ["YATAI_TOKEN"] == '':
    os.environ["YATAI_TOKEN"] = getpass("Enter your YATAI API token: ")

In [232]:
SPACE_KEY = "fa2cbad"
if 'API_KEY' not in locals():
    API_KEY = getpass("Enter your Arize Api Key: ")
os.environ['ARIZE_SPACE_KEY'] = SPACE_KEY
os.environ['ARIZE_API_KEY'] = API_KEY

In [234]:
# login to yatai server

! bentoml yatai login --api-token "$YATAI_TOKEN" --endpoint "http://$YATAI_HOST"

Overriding existing Yatai context config: default
Successfully logged in as user "zuehlke" in organization "default".


In [235]:
# model train

training_df = pd.read_parquet(f"../data/loan_table_training.parquet") # load train dataset
target = "loan_status"
x = training_df[
    training_df.columns.drop(target)
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("zipcode")
    .drop("dob_ssn")
]
X = x.reindex(sorted(x.columns), axis=1)
y = training_df.loc[:, target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=1234
)
classifier = tree.DecisionTreeClassifier()
classifier.fit(X_train[sorted(X_train)], y_train)

DecisionTreeClassifier()

In [236]:
# save model locally to build bento

bentoml.sklearn.save_model("credit_score", classifier)

Model(tag="credit_score:ohqovqe4wk5nuwug", path="/Users/sotnich/bentoml/models/credit_score/ohqovqe4wk5nuwug/")

In [237]:
%%writefile service.py
import bentoml
import pandas as pd
from bentoml.io import PandasDataFrame, JSON
from bentoml._internal.context import trace_context     # hack to return prediction_id
from typing import Dict, Any

bentoml.load_config("bentoml_configuration.yaml")

credit_score_runner = bentoml.sklearn.get("credit_score:latest").to_runner()
model_version = credit_score_runner.models[0].info.version

svc = bentoml.Service("credit_score", runners=[credit_score_runner])

@svc.api(input=PandasDataFrame(), output=JSON())
def classify(input_arr: pd.DataFrame) -> Dict[str, Any]:
    with bentoml.monitor("credit_score_prediction") as mon:
        for feature in input_arr:
            mon.log(input_arr[feature][0], name=feature, role="feature", data_type="numerical")

        result = credit_score_runner.run(input_arr)

        mon.log(str(result[0]), name="pred", role="prediction", data_type="categorical")

    request_id = trace_context.request_id
    return {"prediction": result[0], "prediction_id": request_id, "model_version": model_version}

Writing service.py


In [238]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
- "bentoml_configuration.yaml"
python:
  packages:
    - scikit-learn
    - pandas
    - bentoml-plugins-arize

Writing bentofile.yaml


In [239]:
%%writefile bentoml_configuration.yaml
monitoring:
  enabled: true
  type: bentoml_plugins.arize.ArizeMonitor
  options:
    space_key: SPACE_KEY
    api_key: API_KEY

Writing bentoml_configuration.yaml


In [240]:
! sed -i "" "s/SPACE_KEY/$ARIZE_SPACE_KEY/g" bentoml_configuration.yaml

In [241]:
! sed -i "" "s/API_KEY/$ARIZE_API_KEY/g" bentoml_configuration.yaml

In [242]:
# build bento locally to push it then to bentoml yatai server

!bentoml build

Applying user config override from path: bentoml_configuration.yaml
Building BentoML service "credit_score:sppomge4wkjzcwug" from build context "/Users/sotnich/Sources/mltest/notebooks".
Packing model "credit_score:ohqovqe4wk5nuwug"
Locking PyPI package versions.

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░���░░╚════╝░╚═╝░░░░░╚═╝╚══════╝

Successfully built Bento(tag="credit_score:sppomge4wkjzcwug").


In [243]:
# pushing bento to server
# Then you need to go to YATAI UI and make a deployment

! bentoml push credit_score

╭──────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:… Registering model "credit_score:ohqovqe4wk5nuwug" with Yatai.. .   │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:… Registering model "credit_score:ohqovqe4wk5nuwug" with Yatai.. .   │
╭──────────────────────────────────────────────────────────────────────────────╮
╰──────────────────────────────────────────────────────────────────────────────╯
Uploading model "credit_score:ohqovqe4wk5nuwug" ━━━━ 100.0% • 29.… • 281… • 0:0…
╭──────────────────────────────────────────────────────────────────────────────╮
╰──────────────────────────────────────────────────────────────────────────────╯
Uploading model "credit_score:ohqovqe4wk5nuwug" ━━━━ 100.0% • 29.… • 281… • 0:0…
╭──────────────────────────────────────────────────────────────────────────────╮
╰───────────────────────────